In [1]:

# !pip install asyncio
# !pip install nest-asyncio
# !pip install pandas
# !pip install tqdm

In [2]:
# API_KEY = "zgJAdEg3lkVxGXyEtM89tOZplOsTi8bv" # Personal
API_KEY = "pqmKVrIjJjkKQMhvRslPapP7QzNV2A1I"# Hackathon
notebook_dir = "C:\\Users\\William\\Documents\\ML\\a16z Hackathon\\A16ZxMistral\\single_image_test"
image_foler = "images"
images_dir = f"{notebook_dir}/{image_foler}"

In [3]:

import os
from mistralai import Mistral
import base64
import asyncio
import nest_asyncio
nest_asyncio.apply()
import time
from tqdm.notebook import tqdm

model = "pixtral-12b-latest"
client = Mistral(api_key=API_KEY)

In [4]:
def encode_image_base64(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode("utf-8")


In [5]:
files = os.listdir(images_dir)
encoded_images = [encode_image_base64(f"{images_dir}/{file}") for file in files]
prompt = """"
You are an expert at analysing images. Please answer each of these questions by looking at the images. Please return a JSON containing either True or False depending on the answer to the question. Ensure your response only contains a valid JSON without extrenuous formatting e.g. no ```.

e.g.
{
	"fire": "False",
	"smoke": "False",
	"police": "False",
	"formations": "False",
	"crowds": "False",
	"crush": "False",
}

fire
Is there fire present in the image?

smoke
Is there smoke present in the image?

police
Is there any police or law enforcement present in the image?

Formations
Is there any people orginised in formations e.g. standing in organised lines?

crowds
Do you see any crowds of people of 20 or more?

crush
Do you see any dense crowds of more than 60 people, that could be dangerous to vunerable people?
"""


In [6]:

async def get_pixtral_reponse(encoded_image, prompt):
    try_number = 0
    while True: 
        try:
            start = time.time()
            chat_response = await asyncio.to_thread(
                client.chat.complete,  # Run blocking call in a separate thread
                model=model,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": f"{prompt}"
                            },
                            {
                                "type": "image_url",
                                "image_url": f"data:image/png;base64,{encoded_image}"
                            },
                        ]
                    },
                ]
            )
            response = chat_response.choices[0].message.content
            end = time.time()
            print(f"Attempt: {try_number} Time {end - start:.2f}s")
            break
        except Exception as e: 
            print(f"Error: {e}, retrying... Attempt: {try_number}")
            try_number += 1
            await asyncio.sleep(1)  # Introduce a short delay before retrying
    return response

async def analyse_images(encoded_images, prompt):
    coroutines = [
        get_pixtral_reponse(encoded_image, prompt) 
        for encoded_image in tqdm(encoded_images)
    ]
    results = await asyncio.gather(*coroutines)
    return results

In [7]:
single_result = await get_pixtral_reponse(encoded_images[0], prompt)
print(single_result)


Attempt: 0 Time 3.54s
```json
{
    "fire": "False",
    "smoke": "False",
    "police": "False",
    "formations": "False",
    "crowds": "True",
    "crush": "True"
}
```


In [8]:
final_results = asyncio.run(analyse_images(encoded_images, prompt))

  0%|          | 0/9 [00:00<?, ?it/s]

Error: Server disconnected without sending a response., retrying... Attempt: 0
Error: Server disconnected without sending a response., retrying... Attempt: 0
Error: Server disconnected without sending a response., retrying... Attempt: 0
Error: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}, retrying... Attempt: 0
Error: [SSL: SSLV3_ALERT_BAD_RECORD_MAC] sslv3 alert bad record mac (_ssl.c:2578), retrying... Attempt: 0
Error: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}, retrying... Attempt: 0
Error: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}, retrying... Attempt: 0
Error: [SSL: SSLV3_ALERT_BAD_RECORD_MAC] sslv3 alert bad record mac (_ssl.c:2578), retrying... Attempt: 1
Error: Server disconnected without sending a response., retrying... Attempt: 1
Attempt: 0 Time 2.66s
Error: Server disconnected without sending a response., retrying... Attempt: 1
Error: Server disconnected without sending a response., 

In [ ]:
# final_results
columns = [prompt["name"] for prompt in prompts]
rows = [file.split(".")[0] for file in files]

import pandas as pd

df = pd.DataFrame(final_results, columns=columns, index=rows)
print(df)


ValueError: Shape of passed values is (54, 1), indices imply (9, 6)